In [2]:
import pandas as pd
import numpy as np

from transformers import pipeline, AutoModelForSequenceClassification, TrainingArguments, AutoTokenizer, TextClassificationPipeline

/Users/selinameyer/Documents/GitHub/Gradio_Chatbot/gradio_bot/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [109]:
turns = pd.read_csv("turns_with_context.csv")
user_turns = turns[["utterance_id", "turn_id", "chat_id", "user_utterance"]].copy()
user_turns = user_turns.sort_values(["chat_id", "turn_id"])

In [110]:
user_turns.dropna(inplace=True)

In [111]:
pipe = pipeline("text-classification", model="selmey/behaviour_change_prefilter_german", tokenizer="deepset/gbert-base")

In [112]:
prefiltering_results = []
prefiltering_scores = []
for index, row in user_turns.iterrows():
    res = pipe(row["user_utterance"])
    prefiltering_results.append(res[0]["label"])
    prefiltering_scores.append(res[0]["score"])


In [113]:
user_turns["prefilter"] = prefiltering_results
user_turns["prefilter_score"] = prefiltering_scores

In [114]:
user_turns.head()
user_turns = user_turns[~user_turns["turn_id"].isin([1, 2, 4])]

In [115]:
user_turns.prefilter.value_counts()

prefilter
Change Related    788
Follow/Neutral    671
Name: count, dtype: int64

In [116]:
### Next: classify all turns, that are not Follow/Neutral with valence, label and sublabel. 
valence_pipe = pipeline("text-classification", model="selmey/behaviour-change-valence-german")
label_pipe = pipeline("text-classification", model="selmey/behaviour-change-labels-german")
sublabel_pipe = pipeline("text-classification", model="selmey/behaviour-change-sublabel-german")

In [117]:
valence = []
label = []
for index, row in user_turns.iterrows():
    if row["prefilter"] == "Follow/Neutral":
        valence.append("Follow/Neutral")
        label.append(np.nan)
    else:
        res = valence_pipe(row["user_utterance"])
        valence.append(res[0]["label"])
        res_label = label_pipe(row["user_utterance"])
        label.append(res_label[0]["label"])   


In [118]:
user_turns["valence_new"] = valence
user_turns["label_new"] = label

In [119]:
user_turns.head()

,utterance_id,turn_id,chat_id,user_utterance,prefilter,prefilter_score,valence_new,label_new
1050,3,3,1,Ich könnte schon viel weiter im Leben sein.,Change Related,0.992290,change,Reason
1052,5,5,1,8,Follow/Neutral,0.998908,Follow/Neutral,NaN
1053,6,6,1,Weil ich jetzt etwas ändern musst. Das habe ic...,Change Related,0.960063,change,Reason
1054,9,7,1,Alltag. Faulheit.,Change Related,0.996258,sustain,Reason
1055,12,8,1,Das wäre sehr schön und eine Verbesserung mein...,Change Related,0.996020,change,Reason


In [121]:
sublabel = []
for index, row in user_turns.iterrows():
    if row["label_new"] == "Reason":
        sublabel_res = sublabel_pipe(row["user_utterance"])
        sublabel.append(sublabel_res[0]["label"])
    else:
        sublabel.append(np.nan)

In [126]:
user_turns["sublabel_new"] = sublabel

user_turns.head()

,utterance_id,turn_id,chat_id,user_utterance,prefilter,prefilter_score,valence_new,label_new,sublabel_new
1050,3,3,1,Ich könnte schon viel weiter im Leben sein.,Change Related,0.992290,change,Reason,General Reason
1052,5,5,1,8,Follow/Neutral,0.998908,Follow/Neutral,NaN,NaN
1053,6,6,1,Weil ich jetzt etwas ändern musst. Das habe ic...,Change Related,0.960063,change,Reason,need
1054,9,7,1,Alltag. Faulheit.,Change Related,0.996258,sustain,Reason,General Reason
1055,12,8,1,Das wäre sehr schön und eine Verbesserung mein...,Change Related,0.996020,change,Reason,General Reason


In [127]:
turns_added = pd.merge(turns, user_turns[["utterance_id", "prefilter", "valence_new", "label_new", "sublabel_new"]], on="utterance_id")
turns_added.head()

,Unnamed: 0,condition,chat_id,user_id,target_behaviour,readiness_to_change_start,utterance_id,turn_id,datetime,user_utterance,...,bot_action,bot_datetime,bot_utterance,user_rating,user_rating_explanation,user_word_count,prefilter,valence_new,label_new,sublabel_new
0,2,framework,62,5592a7f2fdf99b3a9e651fd9,gesünder essen,7,705,3,2023-08-08 15:06:12.220701,Ich möchte abnehmen und denke gesündere Ernähr...,...,NaN,2023-08-08 15:06:15.721791,Wie zuversichtlich bist du auf einer Skala von...,NaN,NaN,10,Change Related,change,Reason,desire
1,4,framework,62,5592a7f2fdf99b3a9e651fd9,gesünder essen,7,716,5,2023-08-08 15:09:14.610184,Ich bin noch unsicher ob der Zeitpunkt gut ist.,...,Continuing the Paragraph,2023-08-08 15:09:23.14756,"Du bist dir unsicher, ob der Zeitpunkt gut ist...",gut/hilfreich,Ich bin beruflich sehr eingespannt.,9,Follow/Neutral,Follow/Neutral,NaN,NaN
2,5,framework,62,5592a7f2fdf99b3a9e651fd9,gesünder essen,7,721,6,2023-08-08 15:11:12.903105,Ich bin beruflich eingespannt.,...,Looking Back,2023-08-08 15:11:22.062503,Du bist beruflich stark eingespannt. Kannst du...,gut/hilfreich,Es geht auf mein Anliegen ein.,4,Follow/Neutral,Follow/Neutral,NaN,NaN
3,6,framework,62,5592a7f2fdf99b3a9e651fd9,gesünder essen,7,725,7,2023-08-08 15:12:33.821335,"Ja, es gab diese Zeit.",...,Digging Deep,2023-08-08 15:12:42.029585,"Es gab also eine Zeit, in der du es geschafft ...",gut/hilfreich,Die Frage ist zielgerichtet,5,Follow/Neutral,Follow/Neutral,NaN,NaN
4,7,framework,62,5592a7f2fdf99b3a9e651fd9,gesünder essen,7,728,8,2023-08-08 15:13:59.112905,Ich war motivierter.,...,Affirmation,2023-08-08 15:14:07.758653,Du warst also motivierter. Du hast die Fähigke...,gut/hilfreich,Es müsste als Frage formuliert sein.,3,Change Related,change,Reason,General Reason


In [129]:
turns_added.sort_values(["chat_id", "turn_id"])

,Unnamed: 0,condition,chat_id,user_id,target_behaviour,readiness_to_change_start,utterance_id,turn_id,datetime,user_utterance,...,bot_action,bot_datetime,bot_utterance,user_rating,user_rating_explanation,user_word_count,prefilter,valence_new,label_new,sublabel_new
780,1050,framework,1,6107ab75c50cfb94fb523279,weniger prokrastinieren,8,3,3,2023-08-07 15:43:48.284858,Ich könnte schon viel weiter im Leben sein.,...,NaN,2023-08-07 15:43:53.12469,Wie wichtig ist dir diese Veränderung auf eine...,NaN,NaN,8,Change Related,change,Reason,General Reason
781,1052,framework,1,6107ab75c50cfb94fb523279,weniger prokrastinieren,8,5,5,2023-08-07 15:44:20.584669,8,...,NaN,2023-08-07 15:44:22.691588,Eine 8. Du stehst dieser Veränderung also insg...,NaN,NaN,1,Follow/Neutral,Follow/Neutral,NaN,NaN
782,1053,framework,1,6107ab75c50cfb94fb523279,weniger prokrastinieren,8,6,6,2023-08-07 15:44:58.752073,Weil ich jetzt etwas ändern musst. Das habe ic...,...,Digging Deep,2023-08-07 15:45:08.503392,"Das klingt so, als ob du schon oft den Impuls ...",gut/hilfreich,"Es trifft genau das zu, was ich erläutert habe.",17,Change Related,change,Reason,need
783,1054,framework,1,6107ab75c50cfb94fb523279,weniger prokrastinieren,8,9,7,2023-08-07 15:46:12.641909,Alltag. Faulheit.,...,Undershooting,2023-08-07 15:46:20.580675,"Es scheint, als ob der Alltag und die Faulheit...",gut/hilfreich,Bringt mich dazu mir Gedanken zu machen.,2,Change Related,sustain,Reason,General Reason
784,1055,framework,1,6107ab75c50cfb94fb523279,weniger prokrastinieren,8,12,8,2023-08-07 15:47:41.744613,Das wäre sehr schön und eine Verbesserung mein...,...,Elaboration,2023-08-07 15:47:53.840951,"Es klingt, als ob du dir eine höhere Lebensqua...",gut/hilfreich,Passt einfach.,9,Change Related,change,Reason,General Reason
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362,487,framework,192,5ee7fbc114d0a60f9b076fb6,weniger prokrastinieren,5,2182,8,2023-08-16 17:48:58.469933,sicher zur Schulzeit,...,Affirmation,2023-08-16 17:49:06.439059,"Es ist beeindruckend, dass du dich an eine Zei...",schlecht/unpassend,"ich fange an, mich mehr und mehr an den Floske...",3,Follow/Neutral,Follow/Neutral,NaN,NaN
363,488,framework,192,5ee7fbc114d0a60f9b076fb6,weniger prokrastinieren,5,2183,9,2023-08-16 17:50:46.379971,das mag so sein.,...,Continuing the Paragraph,2023-08-16 17:50:54.029316,"Es ist gut, dass du das erkennst. Es könnte hi...",gut/hilfreich,vielleicht eine Strategie zur Ursachenfindung,4,Follow/Neutral,Follow/Neutral,NaN,NaN
364,489,framework,192,5ee7fbc114d0a60f9b076fb6,weniger prokrastinieren,5,2184,10,2023-08-16 17:52:28.868004,was für Faktoren?,...,Summary,2023-08-16 17:52:48.365382,Lass mich unser bisheriges Gespräch zusammenfa...,gut/hilfreich,sauber zusammengefassst,3,Follow/Neutral,Follow/Neutral,NaN,NaN
365,490,framework,192,5ee7fbc114d0a60f9b076fb6,weniger prokrastinieren,5,2185,11,2023-08-16 17:54:03.363784,wie komme ich auf die Faktoren?,...,Next Steps,2023-08-16 17:54:05.129717,Verstanden! Bitte verrate mir abschließend noc...,schlecht/unpassend,ich hatte eine Frage gestellt,6,Follow/Neutral,Follow/Neutral,NaN,NaN


In [130]:
turns_added.to_csv("turns_reclassified_with_prefilter_context.csv")

In [3]:
turns_added = pd.read_csv("turns_reclassified_with_prefilter_context.csv")

In [4]:
turns_added.head()

,Unnamed: 0.1,Unnamed: 0,condition,chat_id,user_id,target_behaviour,readiness_to_change_start,utterance_id,turn_id,datetime,...,bot_action,bot_datetime,bot_utterance,user_rating,user_rating_explanation,user_word_count,prefilter,valence_new,label_new,sublabel_new
0,0,2,framework,62,5592a7f2fdf99b3a9e651fd9,gesünder essen,7,705,3,2023-08-08 15:06:12.220701,...,NaN,2023-08-08 15:06:15.721791,Wie zuversichtlich bist du auf einer Skala von...,NaN,NaN,10,Change Related,change,Reason,desire
1,1,4,framework,62,5592a7f2fdf99b3a9e651fd9,gesünder essen,7,716,5,2023-08-08 15:09:14.610184,...,Continuing the Paragraph,2023-08-08 15:09:23.14756,"Du bist dir unsicher, ob der Zeitpunkt gut ist...",gut/hilfreich,Ich bin beruflich sehr eingespannt.,9,Follow/Neutral,Follow/Neutral,NaN,NaN
2,2,5,framework,62,5592a7f2fdf99b3a9e651fd9,gesünder essen,7,721,6,2023-08-08 15:11:12.903105,...,Looking Back,2023-08-08 15:11:22.062503,Du bist beruflich stark eingespannt. Kannst du...,gut/hilfreich,Es geht auf mein Anliegen ein.,4,Follow/Neutral,Follow/Neutral,NaN,NaN
3,3,6,framework,62,5592a7f2fdf99b3a9e651fd9,gesünder essen,7,725,7,2023-08-08 15:12:33.821335,...,Digging Deep,2023-08-08 15:12:42.029585,"Es gab also eine Zeit, in der du es geschafft ...",gut/hilfreich,Die Frage ist zielgerichtet,5,Follow/Neutral,Follow/Neutral,NaN,NaN
4,4,7,framework,62,5592a7f2fdf99b3a9e651fd9,gesünder essen,7,728,8,2023-08-08 15:13:59.112905,...,Affirmation,2023-08-08 15:14:07.758653,Du warst also motivierter. Du hast die Fähigke...,gut/hilfreich,Es müsste als Frage formuliert sein.,3,Change Related,change,Reason,General Reason


In [5]:
code_generator = turns_added[["valence_new", "label_new", "sublabel_new"]].copy()

In [6]:
code_generator.replace({"Commitment": "C", "Taking Steps": "TS", "Reason": "R",
                    "change": "+", "sustain": "-", "Follow/Neutral": "FN", 
                    "need": "n", "desire": "d", "General Reason": "_", "ability":"a"}, inplace=True)
code_generator = code_generator.fillna("")
code_generator["code"] = code_generator["label_new"] + code_generator["sublabel_new"] + code_generator["valence_new"]

In [8]:
turns_added["code"] = code_generator["code"]

In [9]:
turns_added.to_csv("turns_reclassified_with_prefilter_context.csv")